In [14]:
BOX_MIN_SIZES = [[10, 16, 24], [32, 48], [64, 96], [128, 192, 256]]
BOX_STEPS = [8, 16, 32, 64]

print('슝=3')

슝=3


In [15]:
image_sizes = (IMAGE_HEIGHT, IMAGE_WIDTH)
min_sizes = BOX_MIN_SIZES
steps= BOX_STEPS

feature_maps = [
    [math.ceil(image_sizes[0] / step), math.ceil(image_sizes[1] / step)]
    for step in steps
]
feature_maps

[[32, 40], [16, 20], [8, 10], [4, 5]]

In [16]:
boxes = []
for k, f in enumerate(feature_maps):
    for i, j in product(range(f[0]), range(f[1])):
        for min_size in min_sizes[k]:
            s_kx = min_size / image_sizes[1]
            s_ky = min_size / image_sizes[0]
            cx = (j + 0.5) * steps[k] / image_sizes[1]
            cy = (i + 0.5) * steps[k] / image_sizes[0]
            boxes += [cx, cy, s_kx, s_ky]

len(boxes)

18800

In [17]:
pretty_boxes = np.asarray(boxes).reshape([-1, 4])
print(pretty_boxes.shape)
print(pretty_boxes)

(4700, 4)
[[0.0125    0.015625  0.03125   0.0390625]
 [0.0125    0.015625  0.05      0.0625   ]
 [0.0125    0.015625  0.075     0.09375  ]
 ...
 [0.9       0.875     0.4       0.5      ]
 [0.9       0.875     0.6       0.75     ]
 [0.9       0.875     0.8       1.       ]]


In [18]:
def default_box():
    image_sizes = (IMAGE_HEIGHT, IMAGE_WIDTH)
    min_sizes = BOX_MIN_SIZES
    steps= BOX_STEPS
    feature_maps = [
        [math.ceil(image_sizes[0] / step), math.ceil(image_sizes[1] / step)]
        for step in steps
    ]
    boxes = []
    for k, f in enumerate(feature_maps):
        for i, j in product(range(f[0]), range(f[1])):
            for min_size in min_sizes[k]:
                s_kx = min_size / image_sizes[1]
                s_ky = min_size / image_sizes[0]
                cx = (j + 0.5) * steps[k] / image_sizes[1]
                cy = (i + 0.5) * steps[k] / image_sizes[0]
                boxes += [cx, cy, s_kx, s_ky]
    boxes = np.asarray(boxes).reshape([-1, 4])
    return boxes

print('슝=3')

슝=3


In [19]:
def _conv_block(inputs, filters, kernel=(3, 3), strides=(1, 1)):
    block_id = id(inputs)

    if strides == (2, 2):
        x = F.pad(inputs, (1, 1, 1, 1), mode='constant', value=0)  # ZeroPadding2D
        x = nn.Conv2d(inputs.size(1), filters, kernel_size=kernel, stride=strides, padding=0, bias=False)(x)
    else:
        x = nn.Conv2d(inputs.size(1), filters, kernel_size=kernel, stride=strides, padding='same', bias=False)(inputs)

    x = nn.BatchNorm2d(filters)(x)
    return F.relu(x)

In [20]:
class DepthwiseConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, strides=(1, 1)):
        super(DepthwiseConvBlock, self).__init__()
        self.strides = strides

        if strides != (1, 1):
            self.pad = nn.ZeroPad2d((1, 1, 1, 1))
        else:
            self.pad = nn.Identity()

        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=strides, padding=0 if strides != (1, 1) else 1, groups=in_channels, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.pad(x)
        x = self.depthwise(x)
        x = F.relu(self.bn1(x))
        x = self.pointwise(x)
        return F.relu(self.bn2(x))

print('슝=3')

슝=3


In [21]:
class BranchBlock(nn.Module):
    def __init__(self, in_channels, filters):
        super(BranchBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, filters, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(filters, filters, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels, filters * 2, kernel_size=3, padding=1)

    def forward(self, x):
        x1 = F.leaky_relu(self.conv1(x))
        x1 = self.conv2(x1)
        x2 = self.conv3(x)
        x = torch.cat([x1, x2], dim=1)
        return F.relu(x)

print('슝=3')

슝=3


In [22]:
class HeadBlock(nn.Module):
    def __init__(self, in_channels, filters):
        super(HeadBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, filters, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        return self.conv(x)

print('슝=3')

슝=3


In [23]:
def _compute_heads(inputs, num_class, num_cell):
    conf = HeadBlock(inputs.size(1), num_cell * num_class)(inputs)
    conf = conf.permute(0, 2, 3, 1).contiguous().view(inputs.size(0), -1, num_class)

    loc = HeadBlock(inputs.size(1), num_cell * 4)(inputs)
    loc = loc.permute(0, 2, 3, 1).contiguous().view(inputs.size(0), -1, 4)

    return conf, loc

print('슝=3')

슝=3


In [24]:
class SsdModel(nn.Module):
    def __init__(self, image_height, image_width, image_labels):
        super(SsdModel, self).__init__()

        self.base_channel = 16
        self.num_cells = [3, 2, 2, 3]
        self.num_class = len(image_labels)

        self.conv_blocks = nn.ModuleList([
            DepthwiseConvBlock(3, self.base_channel * 4, strides=(1, 1)),
            BranchBlock(self.base_channel * 4, self.base_channel * 8)
        ])

        self.layers = nn.ModuleList([
            DepthwiseConvBlock(self.base_channel * 8, self.base_channel * 16, strides=(2, 2)),
            BranchBlock(self.base_channel * 16, self.base_channel),
            DepthwiseConvBlock(self.base_channel * 16, self.base_channel * 16, strides=(2, 2)),
            BranchBlock(self.base_channel * 16, self.base_channel)
        ])

    def forward(self, x):
        x1 = self.conv_blocks[0](x)
        x2 = self.conv_blocks[1](x1)

        x3 = self.layers[0](x2)
        x3 = self.layers[1](x3)

        x4 = self.layers[2](x3)
        x4 = self.layers[3](x4)

        extra_layers = [x1, x2, x3, x4]

        confs, locs = [], []

        for layer, num_cell in zip(extra_layers, self.num_cells):
            conf, loc = _compute_heads(layer, self.num_class, num_cell)
            confs.append(conf)
            locs.append(loc)

        confs = torch.cat(confs, dim=1)
        locs = torch.cat(locs, dim=1)

        predictions = torch.cat([locs, confs], dim=2)

        return predictions

print('슝=3')

슝=3


In [25]:
model = SsdModel(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_LABELS)
print("the number of model layers: ", len(list(model.modules())))
print(model)

the number of model layers:  33
SsdModel(
  (conv_blocks): ModuleList(
    (0): DepthwiseConvBlock(
      (pad): Identity()
      (depthwise): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3, bias=False)
      (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (pointwise): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BranchBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (layers): ModuleList(
    (0): DepthwiseConvBlock(
      (pad): ZeroPad2d((1, 1, 1, 1))
      (depthwise): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), groups=128, bias=False)
      (bn1): BatchNorm2d(128, eps=1e

In [26]:
class _compute_heads(nn.Module):
    def __init__(self, in_channels, num_cell, out_channels):
        super(_compute_heads, self).__init__()
        self.conv = nn.Conv2d(in_channels, num_cell * out_channels, kernel_size=3, padding=1)
        self.num_cell = num_cell
        self.out_channels = out_channels

    def forward(self, x):
        out = self.conv(x)
        out = out.permute(0, 2, 3, 1).contiguous()
        out = out.view(out.size(0), -1, self.out_channels)
        return out

class YourSsdModel(nn.Module):
    def __init__(self, num_classes=2, input_shape=(3, 300, 300), num_cells=[4, 6, 6, 6]):
        super(YourSsdModel, self).__init__()

        self.num_classes = num_classes
        self.num_cells = num_cells

        self.conv1 = nn.Conv2d(input_shape[0], 64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.conf_layers = nn.ModuleList()
        self.loc_layers = nn.ModuleList()

        for num_cell in num_cells:
            self.conf_layers.append(_compute_heads(256, num_cell, self.num_classes))  # 클래스 예측
            self.loc_layers.append(_compute_heads(256, num_cell, 4))  # 바운딩 박스 좌표 예측

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.pool3(x)

        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)

        confs = []
        locs = []

        for conf_layer, loc_layer in zip(self.conf_layers, self.loc_layers):
            confs.append(conf_layer(x))
            locs.append(loc_layer(x))

        confs = torch.cat(confs, dim=1)  # (batch, total_boxes, num_classes)
        locs = torch.cat(locs, dim=1)  # (batch, total_boxes, 4)

        return confs, locs

print('슝=3')

슝=3


In [27]:
# 모델 불러오기,
mymodel = YourSsdModel()
print(mymodel)

YourSsdModel(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conf_layers): ModuleList(
    (0): _compute_heads(
      (conv): Conv2d(256, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1-3): 3 x _compute_heads(
      (c